In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table as dt
from dash.dependencies import Input, Output, State

import os
import numpy as np
import pandas as pd
from pymongo import MongoClient
from bson.json_util import dumps

#### FIX ME #####
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name #Done
from animalShelter import AnimalShelter





###########################
# Data Manipulation / Model
###########################
# FIX ME change for your username and password and CRUD Python module name #Done
username = "accuser"
password = "Helloworld"
shelter = AnimalShelter(username, password)


# class read method must support return of cursor object 
df = pd.DataFrame.from_records(shelter.read({}))



#########################
# Dashboard Layout / View
#########################
app = JupyterDash('CS-340 Final Project')

#FIX ME Add in Grazioso Salvare’s logo #Done
image_filename = 'GraziosoSalvareLogo.png' # replace with your own image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

#FIX ME Place the HTML image tag in the line below into the app.layout code according to your design
#FIX ME Also remember to include a unique identifier such as your name or date
html.Img(src='data:GrazioSalvareLogo;base64,{}'.format(encoded_image.decode()))

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H1('By Thomas Fiske'))),
    html.Hr(),
    html.Div(
        
#Added code to include the requested filter types
        dcc.RadioItems(
            id = 'filter-type',
            options = [ 
                {'label' : 'Water Rescue', 'value': 'water'},
                {'label' : 'Mountain/Wilderness Rescue', 'value' : 'wild'},
                {'label' : 'Disaster Rescue and Individual Tracking', 'value' : 'disaster'},
                {'label' : 'Reset', 'value' : 'reset'},
                      ],
                value = 'reset'
        )
    ),
    html.Hr(),
    dt.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        # Added code to make dash interactive
         editable = False,
        filter_action ="native",
        sort_mode = "multi",
        column_selectable = False,
        row_selectable = False,
        selected_columns =[10],
        selected_rows = [10],
        page_action = "native",
        page_current = 0,
        page_size = 10,
    ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################



    
@app.callback([Output('datatable-id','data'),
               Output('datatable-id','columns')],
              [Input('filter-type', 'value')])
def update_dashboard(filter_type):
### FIX ME Add code to filter interactive data table with MongoDB queries
        if filter_type == 'water':
            df = pd.DataFrame.from_records(animals.read(
                {
                    'animal_type' : 'Dog',
                    'breed' : {'$in' : ['Labrador Retriever Mix', 'Chesapeake Bay Retriever', 'Newfoundland']},
                    'sex_upon_outcome' : 'Intact Female',
                    'age_upon_outcome_in_weeks' : {'$gte' : 26.0, '$lte' : 156.0}
                }))
        elif filter_type == 'wild':
            df = pd.DataFrame.from_records(animals.read(
                {
                    'animal_type' : 'Dog',
                    'breed' : {'$in' : ['German Shepherd', 'Alaskan Malamute', 'Old English Sheepdog',
                                        'Siberian Husky', 'Rottweiler']},
                    'sex_upon_outcome' : 'Intact Male',
                    'age_upon_outcome_in_weeks' : {'$gte' : 26.0, '$lte' : 156.0}
                }))
        elif filter_type == 'disaster':
            df = pd.DataFrame.from_records(animals.read(
                {
                    'animal_type' : 'Dog',
                    'breed' : {'$in' : ['Doberman Pinscher', 'German Shepherd', 'Golden Retriever', 'Bloodhound',
                                       'Rottweiler']},
                    'sex_upon_outcome' : 'Intact Male',
                    'age_upon_outcome_in_weeks' : {'$gte' : 20.0, '$lte' : 300.0}
                }))
        else: #reset
            df = pd.DataFrame.from_records(animals.read({}))
            
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns]
        data=df.to_dict('records')
        
        
        return (data,columns)

@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_graphs(viewData):
    
    dff =pd.DataFrame.from_dict(viewData)
    return [
        dcc.Graph(
    figure = px.histogram(dff, x = 'breed')
        )]

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])

def update_map(viewData):
# #geolocation chart
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
        dl.TileLayer(id="base-layer-id"),
        # Marker with tool tip and popup
            dl.Marker(position=[30.75,-97.48], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                   html.H1("Animal Name"),
                    html.P(dff.iloc[1,9])
                ])
           ])
        ])
   ]   


app

The Username and Password are: accuser Helloworld
Connection sucessful


ServerSelectionTimeoutError: localhost:54196: [Errno 111] Connection refused